In [44]:
import matplotlib
matplotlib.use("Agg")

import config
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report

import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [25]:
# determine the total number of image paths
totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))

In [27]:
# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=25,
	  zoom_range=0.1,
	  width_shift_range=0.1,
	  height_shift_range=0.1,
	  shear_range=0.2,
	  horizontal_flip=True,
	  fill_mode="nearest")

valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [28]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
    config.TRAIN_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=config.BS
)

# initialize the validation generator
valGen = valAug.flow_from_directory(
    config.VAL_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=config.BS
)

# initialize the testing generator
testGen = valAug.flow_from_directory(
    config.TEST_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=config.BS
)

Found 1539 images belonging to 6 classes.
Found 229 images belonging to 6 classes.
Found 759 images belonging to 6 classes.


In [31]:
# print("[INFO] preparing model...")
baseModel = ResNet50(
    weights="imagenet", 
    include_top=False,
	  input_tensor=Input(shape=(224, 224, 3))
)

# construct the new layer for our problem
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)

# place the new layer
model = Model(inputs=baseModel.input, outputs=headModel)

# freeze all layers except the new one
for layer in baseModel.layers:
    layer.trainable = False

94781440/94765736 [==============================] - 1s 0us/step


In [32]:
# compile the model
opt = Adam(learning_rate=config.INIT_LR, decay=config.INIT_LR / config.NUM_EPOCHS)
model.compile(
    loss="categorical_crossentropy", 
    optimizer=opt,
    metrics=["accuracy"])

# train the model
# print("[INFO] training model...")
H = model.fit(
    trainGen,
    steps_per_epoch=totalTrain // config.BS,
    validation_data=valGen,
    validation_steps=totalVal // config.BS,
    epochs=config.NUM_EPOCHS
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
48/48 [==============================] - 1113s 23s/step - loss: 1.1908 - accuracy: 0.5773 - val_loss: 0.5014 - val_accuracy: 0.8348
Epoch 2/20
48/48 [==============================] - 303s 6s/step - loss: 0.6883 - accuracy: 0.7445 - val_loss: 0.4224 - val_accuracy: 0.8393
Epoch 3/20
48/48 [==============================] - 304s 6s/step - loss: 0.5510 - accuracy: 0.7989 - val_loss: 0.4263 - val_accuracy: 0.8438
Epoch 4/20
48/48 [==============================] - 302s 6s/step - loss: 0.4917 - accuracy: 0.8315 - val_loss: 0.3228 - val_accuracy: 0.8750
Epoch 5/20
48/48 [==============================] - 300s 6s/step - loss: 0.4215 - accuracy: 0.8454 - val_loss: 0.2960 - val_accuracy: 0.8661
Epoch 6/20
48/48 [==============================] - 301s 6s/step - loss: 0.3892 - accuracy: 0.8487 - val_loss: 0.2818 - val_accuracy: 0.8884
Epoch 7/20
48/48 [==============================] - 298s 6s/step - loss: 0.3265 - accuracy: 0.8852 - val_loss: 0.2854 - val_accuracy: 0.8929
Epoch 8/20


In [35]:
# reset the testing generator to make predictions on the data
testGen.reset()
predIdxs = model.predict(testGen, steps=(totalTest // config.BS) + 1)

# for each image, find the index of the label for each image that corresponde to the largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))
# print("[INFO] saving model...")
model.save(config.MODEL_PATH, save_format="h5")

              precision    recall  f1-score   support

   cardboard       0.96      0.89      0.93       120
       glass       0.93      0.77      0.85       149
       metal       0.84      0.93      0.88       118
       paper       0.88      0.94      0.91       185
     plastic       0.84      0.91      0.87       148
       trash       0.74      0.72      0.73        39

    accuracy                           0.88       759
   macro avg       0.87      0.86      0.86       759
weighted avg       0.88      0.88      0.88       759



In [ ]:
# plot the training loss and accuracy
N = config.NUM_EPOCHS

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()